In [1]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Oct 14 12:24:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
print(torch.cuda.is_available())

True


In [7]:
!pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 4.1 MB/s 


In [10]:
# import packages
import os, sys
import scipy.io as io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pylab
from PIL import Image
from torch.autograd import Variable
from tqdm import tqdm
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
gasf = GramianAngularField(image_size=256, method='difference')
mtf = MarkovTransitionField(image_size=256)

In [8]:
# import data
datapath='/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Data for testing/Time Series/'

setting1 = '12k Drive End Bearing Fault Data'
setting2 = '12k Fan End Bearing Fault Data'
setting3 = '48k Drive End Bearing Fault Data'

setting = setting1 # Modify here
path = datapath + setting

In [6]:
# import model
modelpath='/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'

# Models for testing

testmodel1 = 'Ex1_12kDE_withoutDA'
testmodel2 = 'Ex1_12FE_withoutDA'
testmodel3 = 'Ex1_48kDE_withoutDA'
testmodel4 = 'Ex1_12kDE_withDA'
testmodel5 = 'Ex1_12FE_withDA'
testmodel6 = 'Ex1_48kDE_withDA'

testmodel = testmodel1 # Modify here

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Conv2d(2, 8, kernel_size=15, padding=7) #256*256->((65+2)-3)*((65+2)-3)=64*64
        self.conv1 = nn.Conv2d(8, 16, kernel_size=5, padding=2) #128*128->((32+2)-4)*((32+2)-4)=30*30
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=2) #32*32->((32+2)-4)*((32+2)-4)=30*30
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1) #128*128->((32+2)-4)*((32+2)-4)=30*30
        self.conv4 = nn.Conv2d(16, 32, kernel_size=3, padding=1) #32*32->((32+2)-4)*((32+2)-4)=30*30

        self.mp = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fc1 = nn.Linear(32*32*32,1000)
        self.fc2 = nn.Linear(32*32*32,1000)
        self.fc3 = nn.Linear(1000,100)
        self.fc4 = nn.Linear(1000,200)
        self.fc5 = nn.Linear(100,3)
        self.fc6 = nn.Linear(200,4)
        self.sigmod=nn.Sigmoid()
        self.logsoftmax = nn.LogSoftmax()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        out = self.mp(self.conv(x))

        out1 = self.mp(self.conv1(out))
        out1 = self.mp(self.conv3(out1))
        out1 = out1.view(-1)
        out1 = self.sigmod(self.fc1(out1))
        out1 = self.sigmod(self.fc3(out1))
        out1 = self.sigmod(self.fc5(out1))

        out2 = self.mp(self.conv2(out))
        out2 = self.mp(self.conv4(out2))
        out2 = out2.view(-1)
        out2 = self.sigmod(self.fc2(out2))
        out2 = self.sigmod(self.fc4(out2))
        out2 = self.sigmod(self.fc6(out2))

        return out1, out2

device='cuda:0'

model = Net()
# model = torch.load(modelpath + testmodel, map_location='cpu')
model = torch.load(modelpath + testmodel)
model.to(device)


Net(
  (conv): Conv2d(2, 8, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
  (conv1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (tanh): Tanh()
  (fc1): Linear(in_features=32768, out_features=1000, bias=True)
  (fc2): Linear(in_features=32768, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=100, bias=True)
  (fc4): Linear(in_features=1000, out_features=200, bias=True)
  (fc5): Linear(in_features=100, out_features=3, bias=True)
  (fc6): Linear(in_features=200, out_features=4, bias=True)
  (sigmod): Sigmoid()
  (logsoftmax): LogSoftmax(dim=None)
  (softmax): Softmax(dim=-1)
)

In [ ]:
# test

f={'Ball':0,'Inner':1,'Outer':2,'7':0,'14':1,'21':2,'28':3}

def test(model, sample):
    X, Y1, Y2 = sample
    data, target1, target2 = Variable(X.reshape((1,) + X.shape)), Variable(Y1).to(torch.long), Variable(Y2).to(torch.long)
    b_x,b_y1,b_y2 = data.cuda(),target1.cuda(),target2.cuda()
    # b_x, b_y1, b_y2 = data, target1, target2
    b_x = b_x.float()
    out1, out2 = model(b_x)
    out1 = out1.reshape((1, len(out1)))
    out2 = out2.reshape((1, len(out2)))

    _, pred1 = torch.max(out1, 1)
    _, pred2 = torch.max(out2, 1)
    num_correct1_test = (pred1 == b_y1).sum()
    num_correct2_test = (pred2 == b_y2).sum()
    return int(num_correct1_test), int(num_correct2_test)

datafiles = os.listdir(path)
counter1, counter2 = 0, 0
accitem1, accitem2 = 0, 0
accsample1, accsample2 = 0, 0
for t,file in tqdm(enumerate(datafiles)):
    if (t + 1) % 13 == 0:
        if counter1 > 6:
            accitem1 += 1
        if counter2 > 6:
            accitem2 += 1
        counter1, counter2 = 0, 0
    data = io.loadmat(os.path.join(path, file))['data'].reshape(1,-1)
    GAFdata = gasf.fit_transform(data)
    MTFdata = mtf.fit_transform(data)
    # print(data.shape)
    y1, y2, _, _ = file.split('_')
    # if (y1!='Ball' and y1!='Inner') or (y2!='7' and y2!='14'):continue
    Y1=f[y1]
    Y2=f[y2]
    X=[GAFdata,MTFdata]
    X = torch.tensor(X)
    # print(X.shape)
    X = X.reshape((2, 256, 256))
    Y1 = torch.tensor(np.array(Y1)).to(torch.long)
    Y2 = torch.tensor(np.array(Y2)).to(torch.long)
    test1, test2 = test(model, [X,Y1,Y2])
    counter1 += test1
    counter2 += test2
    accsample1 += test1
    accsample2 += test2

# Every 13 256-length samples jointly forcast a 1000-length samples, as we crop in the data processing.
print('Finish. Acc1: {:.6f}, Acc2: {:.6f}'.format(accitem1/((len(datafiles)//13)), accitem2/((len(datafiles)//13))))
print('Finish. Acc1: {:.6f}, Acc2: {:.6f}'.format(accsample1/(len(datafiles)), accsample2/(len(datafiles))))

2582it [30:04,  1.45it/s]